In [1]:
from sklearn.datasets import fetch_openml

# Load Pima Indians Diabetes Database
data = fetch_openml(name="diabetes")
X, y = data.data, data.target


C:\Users\R.N.V Siva Karthik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\datasets\_openml.py:311: UserWarning: Multiple active versions of the dataset matching the name diabetes exist. Versions may be fundamentally different, returning version 1.
  warn(
C:\Users\R.N.V Siva Karthik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\datasets\_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [2]:
from sklearn.preprocessing import StandardScaler


In [3]:
X.sample()

,preg,plas,pres,skin,insu,mass,pedi,age
319,6.0,194.0,78.0,0.0,0.0,23.5,0.129,59.0


In [4]:
y

0      tested_positive
1      tested_negative
2      tested_positive
3      tested_negative
4      tested_positive
            ...       
763    tested_negative
764    tested_negative
765    tested_negative
766    tested_positive
767    tested_negative
Name: class, Length: 768, dtype: category
Categories (2, object): ['tested_negative', 'tested_positive']

In [5]:
y.value_counts()

class
tested_negative    500
tested_positive    268
Name: count, dtype: int64

In [6]:
#y=y.map({'tested_positive':1,'tested_negative':0})


In [7]:
from sklearn.model_selection import StratifiedShuffleSplit
ssp=StratifiedShuffleSplit(n_splits=1,test_size=0.2)
trainids,testids=next(ssp.split(X,y))
xtrain=X.iloc[trainids,:]
ytrain=y.iloc[trainids]
xtest=X.iloc[testids,:]
ytest=y.iloc[testids]

In [8]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=10)
knn.fit(xtrain,ytrain)


KNeighborsClassifier(n_neighbors=10)

In [9]:
preds=knn.predict(xtest)

In [10]:
from sklearn.metrics import (recall_score,precision_score,accuracy_score,f1_score,confusion_matrix)

def metricss(ytest,preds):
    print("accuracy:",accuracy_score(ytest,preds))
    print("precision:",precision_score(ytest,preds,pos_label='tested_negative'))
    print("recall:",recall_score(ytest,preds,pos_label='tested_negative'))
    print("f1 score:",f1_score(ytest,preds,pos_label='tested_negative'))
    print("confusion matrix:\n",confusion_matrix(ytest,preds))
metricss(ytest,preds)

accuracy: 0.7467532467532467
precision: 0.7699115044247787
recall: 0.87
f1 score: 0.8169014084507041
confusion matrix:
 [[87 13]
 [26 28]]


In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
pipe=Pipeline([('knn',KNeighborsClassifier())])
fin={
    'knn__n_neighbors':[1,2,3,4,5,10,15]
}
gsv_knn=GridSearchCV(pipe,fin,n_jobs=2,cv=5)
gsv_knn.fit(xtrain,ytrain)
metricss(gsv_knn.predict(xtest),ytest)


accuracy: 0.7922077922077922
precision: 0.88
recall: 0.8148148148148148
f1 score: 0.8461538461538461
confusion matrix:
 [[88 20]
 [12 34]]


In [12]:
gsv_knn.best_params_

{'knn__n_neighbors': 15}

In [13]:
gsv_knn.best_score_

0.734519525523124

In [46]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
pipe=Pipeline([('ran',RandomForestClassifier())])
fin={
    #'ran__n_estimators':[50,100,200,300],
    #'ran__max_depth':[5,10,12,14,16,18,20,30],
    'ran__max_samples':[0.1,0.2,0.3,0.4,0.5,0.6]
}
gsv_ran=GridSearchCV(pipe,fin,n_jobs=2,cv=5)
gsv_ran.fit(xtrain,ytrain)
metricss(gsv_ran.predict(xtest),ytest)

accuracy: 0.7402597402597403
precision: 0.84
recall: 0.7777777777777778
f1 score: 0.8076923076923077
confusion matrix:
 [[84 24]
 [16 30]]


In [44]:
gsv_ran.best_params_

{'ran__max_samples': 0.2}

In [45]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
pipe=Pipeline([('gbc',GradientBoostingClassifier(learning_rate=0.01))])
fin={
    'gbc__n_estimators':[50,100,200,300],
    'gbc__subsample':[0.1,0.2,0.3]
}
gsv_gbc=GridSearchCV(pipe,fin,n_jobs=2,cv=5)
gsv_gbc.fit(xtrain,ytrain)
metricss(gsv_gbc.predict(xtest),ytest)

accuracy: 0.7532467532467533
precision: 0.94
recall: 0.746031746031746
f1 score: 0.831858407079646
confusion matrix:
 [[94 32]
 [ 6 22]]


In [17]:
gsv_gbc.best_params_

{'gbc__n_estimators': 50, 'gbc__subsample': 0.3}

In [18]:
gsv_gbc.best_score_

0.7818072770891644

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
fin={ 'C': [0.1, 1, 10, 100, 1000],  
   'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
   'kernel': ['rbf'] 
   }
gsv_gbc=GridSearchCV(SVC(),fin,n_jobs=2,cv=5)
gsv_gbc.fit(xtrain,ytrain)
metricss(gsv_gbc.predict(xtest),ytest)

accuracy: 0.7597402597402597
precision: 0.89
recall: 0.7739130434782608
f1 score: 0.8279069767441859
confusion matrix:
 [[89 26]
 [11 28]]
